<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/Async_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install asyncio

In [ ]:
async def my_function():
    await some_other_function()


In [4]:
import asyncio

async def greet():
    print("Hi, waiting for 2 seconds...")
    await asyncio.sleep(2)
    print("Done waiting!")

# Run the async function
await greet()


Hi, waiting for 2 seconds...
Done waiting!


In [ ]:
async def task(name, delay):
    print(f"{name} started...")
    await asyncio.sleep(delay)
    print(f"{name} finished after {delay} seconds")

# Run them together
await asyncio.gather(
    task("Task 1", 2),
    task("Task 2", 3),
    task("Task 3", 1),
)


Task 1 started...
Task 2 started...
Task 3 started...
Task 3 finished after 1 seconds
Task 1 finished after 2 seconds
Task 2 finished after 3 seconds


[None, None, None]

In [ ]:
import httpx

async def fetch_joke():
    async with httpx.AsyncClient() as client:
        res = await client.get("https://official-joke-api.appspot.com/jokes/random")
        print(res.json())

await fetch_joke()


/usr/local/lib/python3.11/dist-packages/httpx/_client.py:13: RuntimeWarning: coroutine 'say_hello' was never awaited
  from ._auth import Auth, BasicAuth, FunctionAuth


{'type': 'general', 'setup': 'Why did the mushroom get invited to the party?', 'punchline': 'Because he was a fungi.', 'id': 35}


In [ ]:
async def fetch_multiple_jokes(n=3):
    async with httpx.AsyncClient() as client:
        tasks = [client.get("https://official-joke-api.appspot.com/jokes/random") for _ in range(n)]
        responses = await asyncio.gather(*tasks)
        for i, res in enumerate(responses):
            print(f"Joke {i+1}: {res.json()['setup']} - {res.json()['punchline']}")

await fetch_multiple_jokes(5)


Joke 1: What did the mountain climber name his son? - Cliff.
Joke 2: Why are oranges the smartest fruit? - Because they are made to concentrate. 
Joke 3: I started a new business making yachts in my attic this year... - The sails are going through the roof.
Joke 4: What do you call a fly without wings? - A walk.
Joke 5: Knock knock. 
 Who's there? 
 Hatch. 
 Hatch who? - Bless you!


In [5]:
# prompt: more async functions

async def countdown(name, count):
    print(f"{name}: Starting countdown from {count}...")
    while count > 0:
        print(f"{name}: {count}...")
        await asyncio.sleep(0.5)
        count -= 1
    print(f"{name}: Countdown finished!")

# Run multiple countdowns concurrently
await asyncio.gather(
    countdown("Timer 1", 5),
    countdown("Timer 2", 3),
    countdown("Timer 3", 7)
)

async def fetch_data(url, delay):
    print(f"Fetching data from {url} in {delay} seconds...")
    await asyncio.sleep(delay)
    # Simulate fetching data
    data = f"Data from {url}"
    print(f"Finished fetching data from {url}")
    return data

# Run multiple data fetching tasks
results = await asyncio.gather(
    fetch_data("http://example.com/api/data1", 3),
    fetch_data("http://example.com/api/data2", 2),
    fetch_data("http://example.com/api/data3", 1),
)

print("\nAll data fetched:")
for res in results:
  res


Timer 1: Starting countdown from 5...
Timer 1: 5...
Timer 2: Starting countdown from 3...
Timer 2: 3...
Timer 3: Starting countdown from 7...
Timer 3: 7...
Timer 1: 4...
Timer 2: 2...
Timer 3: 6...
Timer 1: 3...
Timer 2: 1...
Timer 3: 5...
Timer 1: 2...
Timer 2: Countdown finished!
Timer 3: 4...
Timer 1: 1...
Timer 3: 3...
Timer 1: Countdown finished!
Timer 3: 2...
Timer 3: 1...
Timer 3: Countdown finished!
Fetching data from http://example.com/api/data1 in 3 seconds...
Fetching data from http://example.com/api/data2 in 2 seconds...
Fetching data from http://example.com/api/data3 in 1 seconds...
Finished fetching data from http://example.com/api/data3
Finished fetching data from http://example.com/api/data2
Finished fetching data from http://example.com/api/data1

All data fetched:


In [7]:
# prompt: more examples

async def produce(queue, n):
    for i in range(n):
        item = f"item {i+1}"
        print(f"Producing {item}...")
        await asyncio.sleep(0.5)
        await queue.put(item)
    await queue.put(None)  # Signal consumer that production is done

async def consume(queue):
    while True:
        item = await queue.get()
        if item is None:
            print("Consumer: All items processed.")
            break
        print(f"Consuming {item}...")
        await asyncio.sleep(1)
        queue.task_done()

async def run_producer_consumer():
    queue = asyncio.Queue()
    producer_task = asyncio.create_task(produce(queue, 5))
    consumer_task = asyncio.create_task(consume(queue))

    await asyncio.gather(producer_task, consumer_task)

await run_producer_consumer()

async def chain_tasks():
    async def step1():
        print("Step 1 starting...")
        await asyncio.sleep(1)
        print("Step 1 finished.")
        return "result from step 1"

    async def step2(data):
        print(f"Step 2 starting with data: {data}...")
        await asyncio.sleep(1.5)
        print("Step 2 finished.")
        return f"result from step 2 based on {data}"

    async def step3(data):
        print(f"Step 3 starting with data: {data}...")
        await asyncio.sleep(0.8)
        print("Step 3 finished.")
        return f"final result based on {data}"

    result1 = await step1()
    result2 = await step2(result1)
    final_result = await step3(result2)
    print(f"\nChaining complete. Final result: {final_result}")

await chain_tasks()

async def handle_connection(reader, writer):
    addr = writer.get_extra_info('peername')
    print(f"Accepted connection from {addr}")

    while True:
        data = await reader.read(100)  # Read up to 100 bytes
        if not data:
            print(f"Client {addr} disconnected.")
            break

        message = data.decode().strip()
        print(f"Received from {addr}: {message}")

        response = f"Echo: {message}"
        writer.write(response.encode())
        await writer.drain()  # Ensure the response is sent

    writer.close()
    await writer.wait_closed()

async def start_server():
    server = await asyncio.start_server(
        handle_connection, '127.0.0.1', 8888
    )

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()

# Note: Running a server like this will block the cell execution.
# You would typically run this in a separate thread or process,
# or in a non-interactive environment.
# To run this example in Colab, you might need to wrap it
# in a try/except to handle interruption or run it in a
# background process if you want the notebook to remain responsive.
# For demonstration purposes, we will not run the server here directly
# in this Colab cell.

print("\n--- Server example (not run) ---")
print("To run the server, you would execute: await start_server()")
print("This cell will not start the server to avoid blocking.")

async def wait_for_event():
    event = asyncio.Event()

    async def waiter(name, event):
        print(f"Waiter {name}: Waiting for event...")
        await event.wait()
        print(f"Waiter {name}: Event received!")

    async def setter(event):
        print("Setter: Waiting 3 seconds before setting event...")
        await asyncio.sleep(3)
        print("Setter: Setting event!")
        event.set()

    # Run multiple waiters waiting for the same event
    await asyncio.gather(
        waiter("A", event),
        waiter("B", event),
        waiter("C", event),
        setter(event)
    )

await wait_for_event()


Producing item 1...
Producing item 2...
Consuming item 1...
Producing item 3...
Consuming item 2...
Producing item 4...
Producing item 5...
Consuming item 3...
Consuming item 4...
Consuming item 5...
Consumer: All items processed.
Step 1 starting...
Step 1 finished.
Step 2 starting with data: result from step 1...
Step 2 finished.
Step 3 starting with data: result from step 2 based on result from step 1...
Step 3 finished.

Chaining complete. Final result: final result based on result from step 2 based on result from step 1

--- Server example (not run) ---
To run the server, you would execute: await start_server()
This cell will not start the server to avoid blocking.
Waiter A: Waiting for event...
Waiter B: Waiting for event...
Waiter C: Waiting for event...
Setter: Waiting 3 seconds before setting event...
Setter: Setting event!
Waiter A: Event received!
Waiter B: Event received!
Waiter C: Event received!
